Uses:
- bedtools
- gffread
- seqtk
- ushuffle

In [2]:
import os, sys
from gtf import GTF
import glob
import random
import math

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from transforkmers.fasta import Fasta
from scripts.extract_regions import get_intervals

SEQLEN = 1024
RAWDIR = "../data/01_raw/"
INTDIR = f"../data/02_intermediate/{SEQLEN}/"
PRODIR = f"../data/03_processed/{SEQLEN}/"
EXTREMITY = "5prime" # 3prime

In [3]:
!ls {RAWDIR}

Can_Fam4.UU_Cfam_GSD_1.0.fa
Can_Fam4.UU_Cfam_GSD_1.0.fa.fai
Can_Fam4.UU_Cfam_GSD_1.0.gtf
Homo_sapiens.GRCh38.101.gtf
Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa
Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.fai
Mus_musculus.GRCm39.104.gtf
Mus_musculus.GRCm39.dna_sm.primary_assembly.fa
Mus_musculus.GRCm39.dna_sm.primary_assembly.fa.fai


First we associate each annotation (`gtf`) with its genome (`fa`) file.

In [3]:
files = [
    ( RAWDIR + "Can_Fam4.UU_Cfam_GSD_1.0.gtf", RAWDIR + "Can_Fam4.UU_Cfam_GSD_1.0.fa" ),
    ( RAWDIR + "Mus_musculus.GRCm39.104.gtf", RAWDIR + "Mus_musculus.GRCm39.dna_sm.primary_assembly.fa" ),
    ( RAWDIR + "Homo_sapiens.GRCh38.101.gtf",  RAWDIR + "Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa" )
]

# Extract first exon position of GTFs

Next, we iterate over GTFs, and extract only the first transcript of each lncRNA/mRNA gene with only one exon.
Then, we use gffread to extract their nucleotid sequences.

In [13]:
os.makedirs(INTDIR, exist_ok=True)

biotypes = set(["lncRNA", "protein_coding", "processed_transcript",
                "retained_intron", "antisense", "non_coding", "TEC"])

for gtf, fa in files:
    sample = gtf.split("/")[-1].split(".")[0]
    mono_bed = f"{INTDIR}/{sample}.{EXTREMITY}.bed"
    
    with open(gtf) as fd:
        transcripts = []
        for gene in GTF.parse(fd):
            exts = set()
            
            for transcript in gene.transcripts:
                # Check biotype
                if transcript["transcript_biotype"] not in biotypes:
                    continue
                
                # Extract good extremity based on strand and desired extremity
                if (
                    (EXTREMITY == "3prime" and transcript.strand == "+") 
                    or 
                    (EXTREMITY == "5prime" and transcript.strand == "-")
                ):
                        ext = transcript.end
                elif (
                    (EXTREMITY == "3prime" and transcript.strand == "-")
                    or
                    (EXTREMITY == "5prime" and transcript.strand == "+")
                ):
                        ext = transcript.start
                
                # Check if extremity not already seen in other isoforms
                if ext not in exts:
                    exts.add(ext)
                else:
                    continue
                
                transcripts.append(transcript)
        
    intervals = get_intervals(transcripts, SEQLEN, int(EXTREMITY[0]))
    
    with open(mono_bed, "w") as wr:
        for interval in intervals:
            wr.write("\t".join(map(str, interval)) + "\n")

    !bedtools getfasta -name -fi {fa} -bed {mono_bed} | tr a-z A-Z > {INTDIR + sample}.{EXTREMITY}.fa

XM_038588931 skipped: Start Coordinate detected that is < 0.
XM_038589022 skipped: Start Coordinate detected that is < 0.
XM_038589172 skipped: Start Coordinate detected that is < 0.
XM_038589283 skipped: Start Coordinate detected that is < 0.
XM_038589372 skipped: Start Coordinate detected that is < 0.
XR_005387195 skipped: Start Coordinate detected that is < 0.
XM_038589427 skipped: Start Coordinate detected that is < 0.
XM_038589461 skipped: Start Coordinate detected that is < 0.
XR_005387314 skipped: Start Coordinate detected that is < 0.
XR_005387315 skipped: Start Coordinate detected that is < 0.
XR_005387316 skipped: Start Coordinate detected that is < 0.
XM_038589530 skipped: Start Coordinate detected that is < 0.
XM_038589538 skipped: Start Coordinate detected that is < 0.
XM_038589591 skipped: Start Coordinate detected that is < 0.
XR_005387382 skipped: Start Coordinate detected that is < 0.
XM_038589650 skipped: Start Coordinate detected that is < 0.
XR_005387502 skipped: St

Feature (chrUn_JAAHUQ010000447v1:28826-29850) beyond the length of chrUn_JAAHUQ010000447v1 size (29740 bp).  Skipping.
Feature (chrUn_JAAHUQ010000447v1:28825-29849) beyond the length of chrUn_JAAHUQ010000447v1 size (29740 bp).  Skipping.
Feature (chrUn_JAAHUQ010000570v1:27205-28229) beyond the length of chrUn_JAAHUQ010000570v1 size (27726 bp).  Skipping.
Feature (chrUn_JAAHUQ010000583v1:26931-27955) beyond the length of chrUn_JAAHUQ010000583v1 size (27445 bp).  Skipping.
Feature (chrUn_JAAHUQ010001124v1:108363-109387) beyond the length of chrUn_JAAHUQ010001124v1 size (109253 bp).  Skipping.
Feature (chrUn_JAAHUQ010001230v1:86319-87343) beyond the length of chrUn_JAAHUQ010001230v1 size (87233 bp).  Skipping.
Feature (chrUn_JAAHUQ010001563v1:60842-61866) beyond the length of chrUn_JAAHUQ010001563v1 size (61621 bp).  Skipping.
Feature (chrUn_JAAHUQ010001568v1:60894-61918) beyond the length of chrUn_JAAHUQ010001568v1 size (61673 bp).  Skipping.
Feature (chrUn_JAAHUQ010001575v1:60625-61649)

At this step, we have, for each specie, a fasta file containing the sequences labeled `True`. We can combine the differents samples together.

In [8]:
!cat {INTDIR}/*.{EXTREMITY}.fa > {INTDIR}/species.{EXTREMITY}.fa

In [9]:
!grep -c ">" {INTDIR}/species.{EXTREMITY}.fa

398757


In [5]:
VALIDATED = "/groups/dog/tderrien/matthias/validTSS/Can_Fam4.5prime_validCAGE1tpm_1rep.gtf"
!gffread {VALIDATED} -w - -g {FALSE_FA} | tr a-z A-Z > {INTDIR}/{PREFIX}.cage.5prime.fa

# Create false gene sequences

In [10]:
# Dog
#TRUE_FA = f"{INTDIR}/Can_Fam4.5prime.fa"
#PREFIX = "cf4"
#REF_GTF = RAWDIR + "Can_Fam4.UU_Cfam_GSD_1.0.gtf"
#REF_FA = RAWDIR + "Can_Fam4.UU_Cfam_GSD_1.0.fa"
#CHROM_URL = "https://hgdownload.soe.ucsc.edu/goldenPath/canFam4/bigZips/canFam4.chrom.sizes"
#GAP_URL = "https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=1348135893_FTFAH4yg3MzSgyQwThJE0XIXxvZR&boolshad.hgta_printCustomTrackHeaders=0&hgta_ctName=tb_gap&hgta_ctDesc=table+browser+query+on+gap&hgta_ctVis=pack&hgta_ctUrl=&fbQual=whole&fbUpBases=200&fbDownBases=200&hgta_doGetBed=get+BED"

# Human
TRUE_FA = f"{INTDIR}/Homo_sapiens.5prime.fa"
PREFIX = "hsa"
REF_GTF = RAWDIR + "Homo_sapiens.GRCh38.101.gtf"
REF_FA = RAWDIR + "Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa"
CHROM_URL="http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes"
GAP_URL="https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=1299645359_OnnDUejIEZ3YloLAAPZTeauGP1Ra&boolshad.hgta_printCustomTrackHeaders=0&hgta_ctName=tb_gap&hgta_ctDesc=table+browser+query+on+gap&hgta_ctVis=pack&hgta_ctUrl=&fbQual=whole&fbUpBases=200&fbDownBases=200&hgta_doGetBed=get+BED"

## Shuffle true gene sequences

In [11]:
with (
    open(TRUE_FA) as fd, 
    open(f"{INTDIR}/{PREFIX}.shuffled.species.{EXTREMITY}.fa", "w") as wr, 
    open(f"{INTDIR}/{PREFIX}.sequences_length.bed", "w") as bed
):
    fa = Fasta(fd)
    for i, seq in enumerate(fa):
        if i%3 == 0: # Shuffle sequence
            seq.id = f"{seq.id}_shuffled"
            seq.seq = ''.join(random.sample(seq.seq, len(seq)))
            wr.writelines(seq.write())
        else: # Extract length for futur random picking sequence
            seq.id = f"{seq.id}_random"
            bed.write(f'1\t0\t{len(seq.seq)}\t{seq.id}\n')

## Pick random DNA sequences

### Exclusion of unwanted regions

#### Exclude gene positions

First, we extract gene positions from reference transcriptome of Homo sapiens and store theme in `bed` format.

In [12]:
!grep -P "\tgene\t" {REF_GTF} | cut -f1,4,5 > {INTDIR}/{PREFIX}.gene_positions.bed

We also exclude region with "N" using cutN from `SEQTK`.

In [13]:
!seqtk cutN -gp10000000 -n1 {REF_FA} > {INTDIR}/{PREFIX}.N_positions.bed

Finally, we extract gap regions.

In [14]:
!wget -qO- "{GAP_URL}" | sed 's/chr//' > {INTDIR}/{PREFIX}.gap_positions.bed #| sed 's/chr//'

We combine these 3 beds together, and sort.

In [15]:
!cat {INTDIR}/{PREFIX}.*_positions.bed | sort -k1,1 -k2,2n | cut -f1-3 > {INTDIR}/{PREFIX}.to_exclude.bed

## Create and extract random sequences

Next, to use `bedtools shuffle` to get random sequences, we need the chrom sizes.

In [16]:
!wget -qO- "{CHROM_URL}" | sed 's/chr//' | grep -v "_\|M\|Y" > {INTDIR}/{PREFIX}.chrom.sizes # | sed 's/chr//' 

In [17]:
!bedtools shuffle \
    -excl {INTDIR}/{PREFIX}.to_exclude.bed \
    -g {INTDIR}/{PREFIX}.chrom.sizes \
    -i {INTDIR}/{PREFIX}.sequences_length.bed > {INTDIR}/{PREFIX}.random_locations.bed

With the location of the new random sequences in the bed file, we can extract bases.

In [18]:
!bedtools getfasta -fi {REF_FA} -bed {INTDIR}/{PREFIX}.random_locations.bed -name+ | tr a-z A-Z > {INTDIR}/{PREFIX}.random.species.fa

## Merge both negative dataset files
Finally, our false dataset will contained shuffled and random picked sequences. We merge each fasta with:

In [19]:
!cat {INTDIR}/{PREFIX}.shuffled.species.{EXTREMITY}.fa {INTDIR}/{PREFIX}.random.species.fa > {INTDIR}/{PREFIX}.false.species.{EXTREMITY}.fa
!grep -c ">" {INTDIR}/{PREFIX}.false.species.{EXTREMITY}.fa

190041
